Protected web forms
-------------------

You can protect a webform with a secret key. This is usefull for debugging a JSON endpoint as this webform will also function as a normal REST endpoint that you can post JSON to. The advantage here, is that you can open it up in the browser with the secret key and play with it. This type of form should not be used for production as a URL containing a secret key can easilly be leaked thus compromising the security of the endpoint.

In [1]:
import bspump
from bspump.http.web.server import *
from bspump.jupyter import *
import asyncio

In [ ]:
auto_pipeline(
    source=lambda app, pipeline: ProtectedWebFormSource(
        app,
        pipeline,
        route="/",
        form_intro="""<p>This is a protected form.</p>""",
        fields=[
            TextField("foo", display="Foo", description="specify the foo"),
            TextField("la", readonly=True, default="laa"),
            IntField("n"),
            TextField("bar", hidden=True),
            FieldSet("hex", fields=[
               ChoiceField("lol", choices=["a","b","c"]),
               CheckboxField("checkbox"),
            ]),
    ]),
    sink=lambda app, pipeline: JSONWebSink(app, pipeline),
    name="ProtectedWebFormSourcePipeline",
)

In [ ]:
await asyncio.sleep(1)
try:
    event["response"] = {
        "msg": event["form"]["foo"],
        "checkbox": event["form"]["hex"]["checkbox"],
        "n": event["form"]["n"],
        "lol": event["form"]["hex"]["lol"],
    }
except e:
    event["response"] = {"error", str(e)}
event